In [1]:
from transformers import pipeline
from nltk import sent_tokenize
import nltk
import torch

c:\Users\anubh\AppData\Local\Programs\Python\Python313\Scripts\Learning\TV Series Analysis System\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

In [3]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anubh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

###LOAD MODEL

In [4]:
model_name="facebook/bart-large-mnli"
device=0 if torch.cuda.is_available() else -1

In [5]:
def load_model (device):
          theme_classifier=pipeline("zero-shot-classification",model=model_name,device=device)
          return theme_classifier

In [6]:
theme_classifier=load_model(device)

c:\Users\anubh\AppData\Local\Programs\Python\Python313\Scripts\Learning\TV Series Analysis System\env\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anubh\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [ ]:
theme_list = ["friendship","hope","sacrifice","battle","self development","betrayal","love","dialogue"]